In [1]:
import spacy
import math
from random import randint
from tqdm import tqdm
nlp = spacy.load('ja_ginza')

In [2]:
mask_ratio = 0.15
span_length = 3

In [3]:
input_list = []
target_list = []
sentences = []
for line in open('/home/chen/QAbotdata/QAbot_corpus.txt', 'r'):
    if line is not None:
        sentences.append(line.strip())

In [4]:
len(sentences)

6874

In [5]:
for i in tqdm(range(len(sentences))):    
    
    doc = nlp(sentences[i])
    mask_token = math.ceil(len(doc)*mask_ratio)
    chunk_num = math.ceil(mask_token/span_length)
    
    assert chunk_num != 0
    
    index_sequence = [i for i in range(len(doc)-span_length+1)]
    for i in range(len(index_sequence)):
        index_sequence[i] = [i for i in range(i,i+span_length)] 
        
    mask_index = []
    while chunk_num > 0:
        index_sequence_length = len(index_sequence)
        masked_spans = index_sequence[randint(0,index_sequence_length-1)]
        mask_index.extend(masked_spans)
        index_id = index_sequence.index(masked_spans)
        del index_sequence[index_id-span_length+1:index_id+span_length]
        chunk_num = chunk_num - 1    
        
    mask_index.sort()
    
    text_sequence = [i for i in range(len(doc))]
    unmask_index = list(set(text_sequence) - set(mask_index))
    
    unmask_index.sort()
    
    input_text = [None] * len(doc)
    for i in unmask_index:
        input_text[i] = doc[i]
        
        
    re_input_text = []
    for i in range(len(input_text)-1):
        if str(input_text[i]) == 'None' and str(input_text[i+1]) == 'None':
            pass
        else:
            re_input_text.append(input_text[i])
        i+=2               

    re_input_text.append(input_text[len(input_text)-1])   
    
    sent_token_id = 0
    for i in range(len(re_input_text)):
        if str(re_input_text[i]) == 'None':
            re_input_text[i] = f'<extra_id_{sent_token_id}>'
            sent_token_id += 1  
            
    inputs = ''.join(str(e) for e in re_input_text)    
    
    target_text = [None] * len(doc)
    for i in mask_index:
        target_text[i] = doc[i]
        
        
    re_target_text = []
    for i in range(len(target_text)-1):
        if str(target_text[i]) == 'None' and str(target_text[i+1]) == 'None':
            pass
        else:
            re_target_text.append(target_text[i])
        i+=2               

    re_target_text.append(target_text[len(target_text)-1])    
    
    sent_token_id = 0
    for i in range(len(re_target_text)):
        if str(re_target_text[i]) == 'None':
            re_target_text[i] = f'<extra_id_{sent_token_id}>'
            sent_token_id += 1 
            
    targets = ''.join(str(e) for e in re_target_text)  
    
    input_list.append(inputs)
    target_list.append(targets)

100%|███████████████████████████████████████| 6874/6874 [02:39<00:00, 43.08it/s]


In [6]:
assert len(input_list) == len(target_list)

In [7]:
input_list[500]

'ごめんなさい、「課題<extra_id_0>書き間違いでした。課題2の<extra_id_1>javaが提出物に含まれていません。課題3は問題ありませんでした。'

In [8]:
target_list[500]

'<extra_id_0>2」の<extra_id_1>ScoreTest3.<extra_id_2>'

In [9]:
input_list[1000]

'<extra_id_0>結果はOKになっているようです'

In [10]:
target_list[1000]

'現在、採点<extra_id_0>'

In [11]:
file_out = '/home/chen/T5_mask_infilling/masked_QAbot_corpus.txt'
with open(file_out, 'w', encoding = 'utf-8') as f:
    for i in range(len(input_list)):
        f.write(input_list[i].strip() + '\t' +target_list[i].strip() + '\n')
     
        
f.close() 